In [ ]:
# example of pix2pix gan for satellite to map image-to-image translation
import numpy as np
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import *
from keras.layers import *
import os
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
import cv2

## DCP_Predictions

In [ ]:
def guided_filter(p, I, r, eps):
    mean_p = cv2.boxFilter(p, -1, (r, r))
    mean_I = cv2.boxFilter(I, -1, (r, r))
    mean_Ip = cv2.boxFilter(I * p, -1, (r, r))
    cov_Ip = mean_Ip - mean_I * mean_p

    mean_II = cv2.boxFilter(I * I, -1, (r, r))
    var_I = mean_II - mean_I * mean_I

    a = cov_Ip / (var_I + eps)
    b = mean_p - a * mean_I

    mean_a = cv2.boxFilter(a, -1, (r, r))
    mean_b = cv2.boxFilter(b, -1, (r, r))

    q = mean_a * I + mean_b
    return q

def dark_channel(im, size):
    b, g, r = cv2.split(im)
    dc = cv2.min(cv2.min(r, g), b)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (size, size))
    dark = cv2.erode(dc, kernel)
    return dark

def atmospheric_light(im, dark):
    [h, w] = im.shape[:2]
    imsz = h * w
    numpx = int(max(imsz / 1000, 1))
    darkvec = dark.reshape(imsz, 1)
    imvec = im.reshape(imsz, 3)

    indices = darkvec.argsort()
    indices = indices[imsz - numpx::]

    atmsum = np.zeros([1, 3])
    for ind in range(1, numpx):
        atmsum = atmsum + imvec[indices[ind]]

    A = atmsum / numpx
    return A

def transmission_estimate(im, A, sz):
    omega = 0.95
    im3 = np.empty(im.shape, im.dtype)

    for ind in range(0, 3):
        im3[:, :, ind] = im[:, :, ind] / A[0, ind]

    transmission = 1 - omega * dark_channel(im3, sz)
    return transmission

def transmission_refine(im, et):
    im = np.uint8(im * 255)  # Convert image to 8-bit depth
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    gray = np.float64(gray) / 255
    r = 60
    eps = 0.0001
    t = guided_filter(gray, et, r, eps)
    return t

def recover(im, t, A, tx=0.1):
    res = np.empty(im.shape, im.dtype)
    t = cv2.max(t, tx)

    for ind in range(0, 3):
        res[:, :, ind] = (im[:, :, ind] - A[0, ind]) / t + A[0, ind]

    return res

def cloud_removal(im):
    im = np.array(im, dtype=np.float64) / 255

    dark = dark_channel(im, 15)
    A = atmospheric_light(im, dark)
    te = transmission_estimate(im, A, 15)
    t = transmission_refine(im, te)
    recovered = recover(im, t, A)

    return recovered

In [ ]:
cloudy_path = "C:/Users/ArrunPersonal/Codes/ISRO_GAN/PostComments_NewWeights/RICE2_TestImages/cloudy_image"
gt_path = "C:/Users/ArrunPersonal/Codes/ISRO_GAN/PostComments_NewWeights/RICE2_TestImages/ground_truth"

test_clear_imgs = []
DCP_preds = []
for i,j in zip(os.listdir(cloudy_path),os.listdir(gt_path)):
    os.chdir(cloudy_path)
    inp_img = cv2.imread(os.path.join(cloudy_path, i))
    os.chdir(gt_path)
    gt_img = cv2.imread(os.path.join(gt_path, j))
    test_clear_imgs.append(gt_img[:,:,::-1])
    pred = cloud_removal(inp_img)
    DCP_preds.append(pred[:,:,::-1])

## P2P Predictions

In [ ]:
input_size = 512
test_cloud_imgs = []
test_clear_imgs = []
test_cloud_dir = "C:/Users/ArrunPersonal/Codes/ISRO_GAN/PostComments_NewWeights/RICE2_TestImages/cloudy_image"
test_clear_dir = "C:/Users/ArrunPersonal/Codes/ISRO_GAN/PostComments_NewWeights/RICE2_TestImages/ground_truth"
names = []
for i in range(len(os.listdir(test_cloud_dir))):
    name = os.listdir(test_cloud_dir)[i]
    names.append(name)
    os.chdir(test_cloud_dir)
    test_cloud_imgs.append(np.array(Image.open(name))/255) 
    os.chdir(test_clear_dir)
    test_clear_imgs.append(np.array(Image.open(name))/255) 

In [ ]:
test_cloud_imgs = np.array(test_cloud_imgs)
test_clear_imgs = np.array(test_clear_imgs)

In [ ]:
p2p_model = tf.keras.models.load_model("C:/Users/ArrunPersonal/Codes/ISRO_GAN/OldRICEModels/RICEandLandSat8models/RICE2_Models/Pix2PixGAN_RICE2_Weights.h5")
p2p_preds = p2p_model.predict(test_cloud_imgs,batch_size = 1)
del p2p_model

## CloudGAN Model

In [ ]:
# pixel shuffle 
def pixel_shuffle(scale):
    '''
    This function implements pixel shuffling.
    ATTENTION: the scale should be bigger than 2, otherwise just returns the input.
    '''
    if scale > 1:
        return lambda x: tf.nn.depth_to_space(x, scale)
    else:
        return lambda x:x

In [ ]:
def add_down_block(x_inp, filters, kernel_size=(3, 3), padding="same", strides=1,r=False):
    x = Conv2D(filters, kernel_size, padding=padding, strides=strides)(x_inp)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    x = BatchNormalization()(x)
    if r:
        # if r=True then we import an (1X1) Conv2D after input layer 
        # in order the dimensions of 2 tensors coincide.
        x_inp = Conv2D(filters,(1,1), padding=padding, strides=strides)(x_inp)
    x = Add()([x,x_inp])
    return x

def add_up_block(x_inp,skip,filters, kernel_size=(3, 3), padding="same", strides=1,upscale_factor=2):
    x = pixel_shuffle(scale=upscale_factor)(x_inp)
    x = Concatenate()([x, skip])
    x = BatchNormalization()(x)
    x = Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    x = Activation('relu')(x)
    x = Activation('relu')(x)
    return x

def add_bottleneck(x_inp,filters, kernel_size=(3, 3), padding="same", strides=1):
    x = Conv2D(filters, kernel_size, padding=padding, strides=strides)(x_inp)
    x = Activation('relu')(x)
    return x

In [ ]:
def CloudGAN():
    """
      Implementing with Keras the Robust UNet Architecture as proposed by
      Xiaodan Hu, Mohamed A. Naiel, Alexander Wong, Mark Lamm, Paul Fieguth
      in "RUNet: A Robust UNet Architecture for Image Super-Resolution"
    """
    inputs = Input((input_size,input_size, 3))
    
    
    down_1 = Conv2D(64,(7,7), padding="same", strides=1)(inputs)
    down_1 = BatchNormalization()(down_1)
    down_1 = Activation('relu')(down_1)
    
    down_2 = MaxPool2D(pool_size=(2,2))(down_1)
    down_2 = add_down_block(down_2,64)
    down_2 = add_down_block(down_2,64)
    down_2 = add_down_block(down_2,64)
    down_2 = add_down_block(down_2,128,r=True)
    
    down_3 = MaxPool2D(pool_size=(2, 2),strides=2)(down_2)
    down_3 = add_down_block(down_3,128)
    down_3 = add_down_block(down_3,128)
    down_3 = add_down_block(down_3,128)
    down_3 = add_down_block(down_3,256,r=True)
    
    down_4 = MaxPool2D(pool_size=(2, 2))(down_3)
    down_4 = add_down_block(down_4,256)
    down_4 = add_down_block(down_4,256)
    down_4 = add_down_block(down_4,256)
    down_4 = add_down_block(down_4,256)
    down_4 = add_down_block(down_4,256)
    down_4 = add_down_block(down_4,512,r=True) 
    
    down_5 = MaxPool2D(pool_size=(2, 2))(down_4)
    down_5 = add_down_block(down_5,512)
    down_5 = add_down_block(down_5,512)
    down_5 = BatchNormalization()(down_5)
    down_5 = Activation('relu')(down_5)
    
    
    bn_1 = add_bottleneck(down_5, 1024)
    bn_2 = add_bottleneck(bn_1, 512)
    
    up_1 = add_up_block(bn_2,down_5, 512,upscale_factor=1)
    up_2 = add_up_block(up_1,down_4, 384,upscale_factor=2)
    up_3 = add_up_block(up_2,down_3, 256,upscale_factor=2)
    up_4 = add_up_block(up_3,down_2, 96,upscale_factor=2) 
    
    up_5 = pixel_shuffle(scale=2)(up_4)
    up_5 = Concatenate()([up_5,down_1])
    up_5 = Conv2D(99,(3,3), padding="same", strides=1)(up_5)
    up_5 = Activation('relu')(up_5)
    up_5 = Conv2D(99,(3,3), padding="same", strides=1)(up_5)
    up_5 = Activation('relu')(up_5)
   
    outputs = Conv2D(3,(1,1), padding="same")(up_5)
    model = Model(inputs, outputs)
    return model

## CloudGAN Predictions

In [ ]:
CloudGAN_model = CloudGAN()
CloudGAN_model.load_weights('C:/Users/ArrunPersonal/Codes/ISRO_GAN/OldRICEModels/RICEandLandSat8models/RICE2_Models/CloudGAN_rice2.h5')
CloudGAN_model_preds = CloudGAN_model.predict(test_cloud_imgs,batch_size = 1)
del CloudGAN_model

## DeCloud GAN Predictions

In [ ]:
input_size = 512
batch_size = 4

In [ ]:
def define_discriminator(image_shape):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# source image input
	in_src_image = Input(shape=image_shape)
	# target image input
	in_target_image = Input(shape=image_shape)
	# concatenate images channel-wise
	merged = Concatenate()([in_src_image, in_target_image])
	# C64
	d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
	d = LeakyReLU(alpha=0.2)(d)
	# C128
	d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C256
	d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C512
	d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# second last output layer
	d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# patch output
	d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
	patch_out = Activation('sigmoid')(d)
	# define model
	model = Model([in_src_image, in_target_image], patch_out)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
	return model
 
# define image shape
image_shape = (input_size,input_size,3)
# create the model
model = define_discriminator(image_shape)
# summarize the model
# model.summary()

In [ ]:
# pixel shuffle 
def pixel_shuffle(scale):
    '''
    This function implements pixel shuffling.
    ATTENTION: the scale should be bigger than 2, otherwise just returns the input.
    '''
    if scale > 1:
        return lambda x: tf.nn.depth_to_space(x, scale)
    else:
        return lambda x:x

In [ ]:
def add_down_block(x_inp, filters, kernel_size=(3, 3), padding="same", strides=1,r=False):
    x = Conv2D(filters, kernel_size, padding=padding, strides=strides)(x_inp)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    x = BatchNormalization()(x)
    if r:
        # if r=True then we import an (1X1) Conv2D after input layer 
        # in order the dimensions of 2 tensors coincide.
        x_inp = Conv2D(filters,(1,1), padding=padding, strides=strides)(x_inp)
    x = Add()([x,x_inp])
    return x

def add_up_block(x_inp,skip,filters, kernel_size=(3, 3), padding="same", strides=1,upscale_factor=2):
    x = pixel_shuffle(scale=upscale_factor)(x_inp)
    x = Concatenate()([x, skip])
    x = BatchNormalization()(x)
    x = Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    x = Activation('relu')(x)
    x = Activation('relu')(x)
    return x

def add_bottleneck(x_inp,filters, kernel_size=(3, 3), padding="same", strides=1):
    x = Conv2D(filters, kernel_size, padding=padding, strides=strides)(x_inp)
    x = Activation('relu')(x)
    return x

In [ ]:
def RUNet():
    """
      Implementing with Keras the Robust UNet Architecture as proposed by
      Xiaodan Hu, Mohamed A. Naiel, Alexander Wong, Mark Lamm, Paul Fieguth
      in "RUNet: A Robust UNet Architecture for Image Super-Resolution"
    """
    inputs = Input((input_size,input_size, 3))
    
    
    down_1 = Conv2D(64,(7,7), padding="same", strides=1)(inputs)
    down_1 = BatchNormalization()(down_1)
    down_1 = Activation('relu')(down_1)
    
    down_2 = MaxPool2D(pool_size=(2,2))(down_1)
    down_2 = add_down_block(down_2,64)
    down_2 = add_down_block(down_2,64)
    down_2 = add_down_block(down_2,64)
    down_2 = add_down_block(down_2,128,r=True)
    
    down_3 = MaxPool2D(pool_size=(2, 2),strides=2)(down_2)
    down_3 = add_down_block(down_3,128)
    down_3 = add_down_block(down_3,128)
    down_3 = add_down_block(down_3,128)
    down_3 = add_down_block(down_3,256,r=True)
    
    down_4 = MaxPool2D(pool_size=(2, 2))(down_3)
    down_4 = add_down_block(down_4,256)
    down_4 = add_down_block(down_4,256)
    down_4 = add_down_block(down_4,256)
    down_4 = add_down_block(down_4,256)
    down_4 = add_down_block(down_4,256)
    down_4 = add_down_block(down_4,512,r=True) 
    
    down_5 = MaxPool2D(pool_size=(2, 2))(down_4)
    down_5 = add_down_block(down_5,512)
    down_5 = add_down_block(down_5,512)
    down_5 = BatchNormalization()(down_5)
    down_5 = Activation('relu')(down_5)
    
    
    bn_1 = add_bottleneck(down_5, 1024)
    bn_2 = add_bottleneck(bn_1, 512)
    
    up_1 = add_up_block(bn_2,down_5, 512,upscale_factor=1)
    up_2 = add_up_block(up_1,down_4, 384,upscale_factor=2)
    up_3 = add_up_block(up_2,down_3, 256,upscale_factor=2)
    up_4 = add_up_block(up_3,down_2, 96,upscale_factor=2) 
    
    up_5 = pixel_shuffle(scale=2)(up_4)
    up_5 = Concatenate()([up_5,down_1])
    up_5 = Conv2D(99,(3,3), padding="same", strides=1)(up_5)
    up_5 = Activation('relu')(up_5)
    up_5 = Conv2D(99,(3,3), padding="same", strides=1)(up_5)
    up_5 = Activation('relu')(up_5)
   
    outputs = Conv2D(3,(1,1), padding="same")(up_5)
    model = Model(inputs, outputs)
    return model

In [ ]:
DeCloud_model = RUNet()
DeCloud_model.load_weights('C:/Users/ArrunPersonal/Codes/ISRO_GAN/OldRICEModels/RICEandLandSat8models/RICE2_Models/DeCloudGAN_max_psnr_rice2.h5')
DeCloud_preds = DeCloud_model.predict(test_cloud_imgs,batch_size = 1)
del DeCloud_model

## RS_CloudGAN Predictions 

In [ ]:
def encoder_block(input_features,num_filters, filter_size=[3,5,7]):
    conv1_1 = Conv2D(num_filters,filter_size[0], padding = 'same',activation = 'relu')(input_features)
    conv1_2 = Conv2D(num_filters,filter_size[1], padding = 'same',activation = 'relu')(input_features)
    conv1_3 = Conv2D(num_filters,filter_size[2], padding = 'same',activation = 'relu')(input_features)

    concat_12 = Concatenate()([conv1_1,conv1_2])
    concat_13 = Concatenate()([conv1_2,conv1_3])
    concat_23 = Concatenate()([conv1_1,conv1_3])

    conv2_1 = Conv2D(num_filters,3, padding = 'same',activation = 'relu')(concat_12)
    conv2_2 = Conv2D(num_filters,3, padding = 'same',activation = 'relu')(concat_23)
    conv2_3 = Conv2D(num_filters,3, padding = 'same',activation = 'relu')(concat_13)

    concat_123 = Concatenate()([conv2_1,conv2_2,conv2_3])
    conv_fin = Conv2D(num_filters,3, padding = 'same',activation = 'relu')(concat_123)
    maxpool_fin = MaxPool2D(2)(conv_fin)
    return maxpool_fin

In [ ]:
def decoder_block(input_layer,skip_connection, num_filters,filter_size=[3,5,7]):
    convt1_1 = Conv2DTranspose(num_filters, filter_size[0], padding = 'same',activation = 'relu')(input_layer)
    convt1_2 = Conv2DTranspose(num_filters, filter_size[1], padding = 'same',activation = 'relu')(input_layer)
    convt1_3 = Conv2DTranspose(num_filters, filter_size[2], padding = 'same',activation = 'relu')(input_layer)

    concat_12 = Concatenate()([convt1_1,convt1_2])
    concat_13 = Concatenate()([convt1_2,convt1_3])
    concat_23 = Concatenate()([convt1_1,convt1_3])

    conv2_1 = Conv2D(num_filters,3, padding = 'same',activation = 'relu')(concat_12)
    conv2_2 = Conv2D(num_filters,3, padding = 'same',activation = 'relu')(concat_23)
    conv2_3 = Conv2D(num_filters,3, padding = 'same',activation = 'relu')(concat_13)

    concat_123 = Concatenate()([conv2_1,conv2_2,conv2_3,skip_connection])
    conv_fin = Conv2D(num_filters,3, padding = 'same',activation = 'relu')(concat_123)
    upsampling_fin = UpSampling2D(2)(conv_fin)
    
    return upsampling_fin

In [ ]:
def bottleneck(input_layer,drop = 0.2):
    
    feature_layer = Conv2D(512,3,padding = 'same',activation = 'linear')(input_layer)
    attention_layer = Conv2D(512,3,padding = 'same',activation = 'sigmoid')(feature_layer)
    new_input_features = MultiHeadAttention(num_heads=3, key_dim=3, attention_axes=(2, 3))(input_layer,attention_layer)
    
    batch_norma = BatchNormalization()(new_input_features)
    if(drop):
        drop = Dropout(drop)(batch_norma)
        return drop
    return batch_norma

In [ ]:
input_layer = Input((512,512,3))
e1 = encoder_block(input_layer,32)
e2 = encoder_block(e1,64)
e3 = encoder_block(e2,128)
e4 = encoder_block(e3,256)
e5 = encoder_block(e4,512)
b = bottleneck(e5)
d1 = decoder_block(b,e5,256)
d2 = decoder_block(d1,e4,256)
d3 = decoder_block(d2,e3,128)
d4 = decoder_block(d3,e2,64)
d5 = decoder_block(d4,e1,3)

#CR_Net = Model(inputs = input_layer, outputs = d5)

In [ ]:
RS_Cloud_model = Model(inputs = input_layer, outputs = d5)
RS_Cloud_model.load_weights('C:/Users/ArrunPersonal/Codes/ISRO_GAN/PostComments_NewWeights/RICE2_weights/RS_CloudGAN_l5_k357_selfattn_bce_max_psnr_rice2.h5')
RS_Cloud_preds = RS_Cloud_model.predict(test_cloud_imgs,batch_size = 1)
del RS_Cloud_model

## SpA GAN predictions

In [ ]:
#!pip install attrdict

In [ ]:
# To predict cloudless images for cloudy test images with pretrained model
#!python /content/drive/MyDrive/Cloud_Removal_using_SpA_GAN/predict.py --config /content/drive/MyDrive/Cloud_Removal_using_SpA_GAN/config.yml --test_dir /content/drive/MyDrive/ISROCodes/RICE1_TestImages --out_dir /content/drive/MyDrive/ISROCodes/RICE1_Models/SpAGAN_Results_Updated/ReLU_Run --pretrained /content/drive/MyDrive/ISROCodes/RICE1_Models/ReLU_gen_model_epoch_200.pth

## Load SpA GAN predicitons

In [ ]:
SpA_GAN_preds = []
os.chdir("C:/Users/ArrunPersonal/Codes/ISRO_GAN/OldRICEModels/RICEandLandSat8models/RICE2_Models/SpAGAN_Results/LeakyReLU_Run/cloudless/epoch_0001")
for i in range(len(os.listdir(os.getcwd()))):
    im_name = f"cloudless_{names[i]}"
    SpA_GAN_preds.append(np.array(Image.open(im_name))/255)

In [ ]:
SpA_GAN_preds = np.array(SpA_GAN_preds)
print(np.shape(SpA_GAN_preds))

## Net Preditions

In [ ]:
num_imgs = 12
for i in range(num_imgs):
    plt.figure(figsize = (48,30))
    plt.subplot(1,8,1).imshow(np.array(test_cloud_imgs[i],dtype = 'float32'))
    plt.title("Cloudy Image",fontsize = 20)
    plt.axis('off')
    plt.subplot(1,8,2).imshow(np.array(test_clear_imgs[i],dtype = 'float32'))
    plt.title("Clear GT Image",fontsize = 20)
    plt.axis('off')
    plt.subplot(1,8,3).imshow(DCP_preds[i])
    plt.title("DCP predictions",fontsize = 20)
    plt.axis('off')
    plt.subplot(1,8,4).imshow(p2p_preds[i])
    plt.title("Pix2Pix GAN predictions",fontsize = 20)
    plt.axis('off')
    plt.subplot(1,8,5).imshow(CloudGAN_model_preds[i])
    plt.title("Cloud GAN predictions",fontsize = 20)
    plt.axis('off')
    plt.subplot(1,8,6).imshow(SpA_GAN_preds[i])
    plt.title("SpA GAN predictions",fontsize = 20)
    plt.axis('off')
    plt.subplot(1,8,7).imshow(DeCloud_preds[i])
    plt.title("DeCloud GAN predictions",fontsize = 20)
    plt.axis('off')
    plt.subplot(1,8,8).imshow(RS_Cloud_preds[i])
    plt.title("RS_Cloud GAN predictions",fontsize = 20)
    plt.axis('off')